# Deterministic Inputs, Noisy “And” gate model (DINA)

This notebook will show you how to train and use the GDDINA.
First, we will show how to get the data (here we use a0910 as the dataset).
Then we will show how to train a DINA and perform the parameters persistence.
At last, we will show how to load the parameters from the file and evaluate on the test dataset.

The script version could be found in [DINA.py](DINA.ipynb)

## Data Preparation

Before we process the data, we need to first acquire the dataset which is shown in [prepare_dataset.ipynb](prepare_dataset.ipynb)

In [2]:
import pandas as pd

# from tests.irr.conftest import knowledge

train_data = pd.read_csv("../../../data/comta/train.csv")
valid_data = pd.read_csv("../../../data/comta/valid.csv")
test_data = pd.read_csv("../../../data/comta/test.csv")
item_data = pd.read_csv("../../../data/comta/item.csv")

knowledge_num = 165


def code2vector(x):
    vector = [0] * knowledge_num
    for k in eval(x):
        vector[k - 1] = 1
        
    return vector


item_data["knowledge"] = item_data["knowledge_code"].apply(code2vector)
item_data.drop(columns=["knowledge_code"], inplace=True)

train_data = pd.merge(train_data, item_data, on="item_id")
valid_data = pd.merge(valid_data, item_data, on="item_id")
test_data = pd.merge(test_data, item_data, on="item_id")

train_data.head(10)


,user_id,item_id,score,knowledge
0,36,189,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,51,239,0,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,101,463,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,101,466,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,34,183,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5,63,301,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6,18,102,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7,60,287,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
8,85,389,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9,121,566,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [3]:
train_data.user_id.nunique()

110

In [4]:
len(train_data['knowledge'][0])

165

In [5]:
len(train_data), len(valid_data), len(test_data)

(297, 178, 120)

In [6]:
# Transform data to torch Dataloader (i.e., batchify)
# batch_size is set to 256
import torch
from torch.utils.data import TensorDataset, DataLoader

batch_size = 1

def transform(x, y, z, k, batch_size, **params):
    dataset = TensorDataset(
        torch.tensor(x, dtype=torch.int64),
        torch.tensor(y, dtype=torch.int64),
        torch.tensor(k, dtype=torch.float32),
        torch.tensor(z, dtype=torch.float32)
    )
    return DataLoader(dataset, batch_size=batch_size, **params)


train, valid, test = [
    transform(data["user_id"], data["item_id"], data["score"], data["knowledge"], batch_size)
    for data in [train_data, valid_data, test_data]
]
train, valid, test


C:\Users\61489\AppData\Local\Temp\ipykernel_27848\775724876.py:10: UserWarning: Failed to initialize NumPy: DLL load failed while importing _multiarray_umath: 找不到指定的模块。 (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_numpy.cpp:68.)
  torch.tensor(x, dtype=torch.int64),


(<torch.utils.data.dataloader.DataLoader at 0x2352c60cdf0>,
 <torch.utils.data.dataloader.DataLoader at 0x23533232fa0>)

## Training and Persistence

In [7]:
import logging
logging.getLogger().setLevel(logging.INFO)

In [10]:
from EduCDM import GDDINA

cdm = GDDINA(4164, 17747, knowledge_num)

cdm.train(train, valid, epoch=100)
cdm.save("dina.params")

Epoch 0: 100%|██████████| 297/297 [00:01<00:00, 262.49it/s]


[Epoch 0] LogisticLoss: 0.709004


evaluating: 100%|██████████| 178/178 [00:00<00:00, 4281.87it/s]


[Epoch 0] auc: 0.478539, accuracy: 0.432584


Epoch 1: 100%|██████████| 297/297 [00:01<00:00, 266.95it/s]


...

[Epoch 98] auc: 0.499543, accuracy: 0.471910


Epoch 99: 100%|██████████| 297/297 [00:02<00:00, 124.18it/s]


[Epoch 99] LogisticLoss: 0.494043


evaluating: 100%|██████████| 178/178 [00:00<00:00, 2103.71it/s]
INFO:root:save parameters to dina.params


[Epoch 99] auc: 0.499543, accuracy: 0.471910


## Loading and Testing

In [11]:
cdm.load("dina.params")
auc, accuracy = cdm.eval(test)
print("auc: %.6f, accuracy: %.6f" % (auc, accuracy))

INFO:root:load parameters from dina.params
evaluating: 100%|██████████| 120/120 [00:00<00:00, 2650.82it/s]

auc: 0.584456, accuracy: 0.583333
